# ⚙️ Initiez-vous au MLOps (partie 1/2)

## 🫁 Detection du cancer du poumon

L'efficacité du système de prédiction du cancer aide les personnes à connaître leur risque de cancer à faible coût et les aide également à prendre la décision appropriée en fonction de leur statut de risque de cancer. Les données sont collectées à partir du site web du système de prédiction en ligne du cancer du poumon.

###  📦 Initialisation du notebook

In [3]:
import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))

import numpy as np
import pandas as pd
from src.visualization.visu_text import quick_df_info

### 📋 Chargement des données brutes

**Nombre total d'attributs : 16**

| N° | Original (Anglais) | Traduction française | Valeurs |
|----|-------------------|---------------------|---------|
| 1  | Gender: M(male), F(female) | Genre : H(homme), F(femme) | M/F ou H/F |
| 2  | Age: Age of the patient | Âge : Âge du patient | Numérique |
| 3  | Smoking: YES=2, NO=1 | Tabagisme : OUI=2, NON=1 | 1 ou 2 |
| 4  | Yellow fingers: YES=2, NO=1 | Doigts jaunes : OUI=2, NON=1 | 1 ou 2 |
| 5  | Anxiety: YES=2, NO=1 | Anxiété : OUI=2, NON=1 | 1 ou 2 |
| 6  | Peer_pressure: YES=2, NO=1 | Influence sociale tabagique : OUI=2, NON=1 | 1 ou 2 |
| 7  | Chronic Disease: YES=2, NO=1 | Maladie chronique : OUI=2, NON=1 | 1 ou 2 |
| 8  | Fatigue: YES=2, NO=1 | Fatigue : OUI=2, NON=1 | 1 ou 2 |
| 9  | Allergy: YES=2, NO=1 | Allergie : OUI=2, NON=1 | 1 ou 2 |
| 10 | Wheezing: YES=2, NO=1 | Sifflement respiratoire : OUI=2, NON=1 | 1 ou 2 |
| 11 | Alcohol: YES=2, NO=1 | Alcool : OUI=2, NON=1 | 1 ou 2 |
| 12 | Coughing: YES=2, NO=1 | Toux : OUI=2, NON=1 | 1 ou 2 |
| 13 | Shortness of Breath: YES=2, NO=1 | Essoufflement : OUI=2, NON=1 | 1 ou 2 |
| 14 | Swallowing Difficulty: YES=2, NO=1 | Difficulté de déglutition : OUI=2, NON=1 | 1 ou 2 |
| 15 | Chest pain: YES=2, NO=1 | Douleur thoracique : OUI=2, NON=1 | 1 ou 2 |
| 16 | Lung Cancer: YES, NO | Cancer du poumon : OUI, NON | OUI/NON |

## Notes
- Les attributs 3-15 utilisent un codage binaire : 1=NON, 2=OUI
- L'attribut 16 (Cancer du poumon) semble être la variable cible du dataset
- Ce dataset paraît être conçu pour la prédiction du cancer du poumon basée sur des facteurs de risque et symptômes


In [17]:
raw_df = pd.read_csv("../data/raw/survey_lung_cancer.csv")
quick_df_info(raw_df, "Données brutes")


┌------------------------------* Données brutes *------------------------------┐
├─------- Shape: (4455, 16) - Colonnes:
├─GENDER                    object    
├─AGE                       int64     
├─SMOKING                   int64     
├─YELLOW_FINGERS            int64     
├─ANXIETY                   int64     
├─PEER_PRESSURE             int64     
├─CHRONIC DISEASE           int64     
├─FATIGUE                   int64     
├─ALLERGY                   int64     
├─WHEEZING                  int64     
├─ALCOHOL CONSUMING         int64     
├─COUGHING                  int64     
├─SHORTNESS OF BREATH       int64     
├─SWALLOWING DIFFICULTY     int64     
├─CHEST PAIN                int64     
├─LUNG_CANCER               object    
└------------------------------------------------------------------------------┘


### 🛠 Transformation des données en binaire 
- GENDER F=0, M=1
- LUNG_CANCER NO=0, YES=1 
- AUTRES 1=0, 2=1

In [18]:
cols = raw_df.columns.tolist()
delete = ["LUNG_CANCER", "GENDER", "AGE"]
cols = [x for x in cols if x not in delete]
for col in cols:
    raw_df[col] = raw_df[col].map({1: 0, 2: 1})
raw_df["GENDER"] = raw_df["GENDER"].map({"F": 0, "M": 1})
raw_df["LUNG_CANCER"] = raw_df["LUNG_CANCER"].map({"NO": 0, "YES": 1})
raw_df.head()

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,LUNG_CANCER
0,1,69,0,1,1,0,0,1,0,1,1,1,1,1,1,1
1,1,74,1,0,0,0,1,1,1,0,0,0,1,1,1,1
2,0,59,0,0,0,1,0,1,0,1,0,1,1,0,1,0
3,1,63,1,1,1,0,0,0,0,0,1,0,0,1,1,0
4,0,63,0,1,0,0,0,0,0,1,0,1,1,0,0,0


### 🧽 Sauvegarde du JDD nettoyé

In [19]:
raw_df.to_csv("../data/processed/survey_lung_cancer_clean.csv")